# Orginsation of EDM tracks with Dimension Reduction

In [ ]:
#Imports
import librosa
import librosa.display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import matplotlib.patches as patches
import os
import csv
import scipy
import sklearn
import sklearn.cluster
import seaborn as sns
import time
import copy
import pyrubberband as pyrb
from ipynb.fs.full.musicalkeyfinder import *
from matplotlib.lines import Line2D
import plotly.express as px
import plotly.graph_objs as go

sns.set(style="darkgrid", palette="muted")
plt.style.use("fivethirtyeight")
plt.rcParams["font.size"] = 12
plt.rcParams["axes.labelsize"] = 13
plt.rcParams["axes.titlesize"] = 13
plt.rcParams["figure.figsize"] = 16, 10
plt.rcParams["figure.dpi"] = 100
plt.rcParams["xtick.labelsize"] = 11
plt.rcParams["ytick.labelsize"] = 11
plt.rcParams["legend.fontsize"] = 11

## Internal utility functions

### Feature Extractor

In [ ]:
def extractTrackDataAndWriteToCSV(y, sr, output_csv):

    #----------------------------------------------------------------------------
    #-- Preperations --#
    hop_length = 512
    
    #seperate harmonics from percussives
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    # Musical-key-extractor by jackmcarthur (git-project)
    keyextract = Tonal_Fragment(y_harmonic, sr=sr)   
    # Separate a complex-valued spectrogram into its magnitude and phase
    S, phase = librosa.magphase(librosa.stft(y))
    # Compute onset strengths for tempo estimation purposes
    oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
    
    #----------------------------------------------------------------------------
    #-- Extraction --#    
    
    # Extract tempo - beats per minute
    tempo = float(librosa.beat.tempo(onset_envelope=oenv, sr=sr))
    # Extract onsets and devide by length to get rate
    onset = librosa.onset.onset_detect(y=y, sr=sr)
    onset_rate = (np.sum(onset) / np.size(onset))
    # Extract zero crossings rate
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=y))
    # Extract spectral centroid
    spec_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    # Extract spectral contrast
    spec_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr)) 
    # Extract spectral rolloff
    spec_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))   
    # Extract spectral_bandwidth -
    spec_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))  
    # Extract tonnetz - 
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr, chroma=None)
    # Extract MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    
    #----------------------------------------------------------------------------
    #-- Processing --# 
    
    # Appends mean values of all arrays of data
    to_append = f'{tempo:5.3f} {onset_rate:5.3f} {zcr:5.3f} {spec_centroid:5.3f} {spec_contrast:5.3f} {spec_rolloff:5.3f} {spec_bandwidth:5.3f}'
    
    # Add tonnetz
    for t in tonnetz:
        to_append += f' {np.sum(t):5.3f}'
        
    # Add chroma tones
    keyextract.chroma_max = max(keyextract.chroma_vals)
    for key, chrom in keyextract.keyfreqs.items():
        to_append += f' {(chrom / keyextract.chroma_max):5.3f}'
        
    # Add mfccs
    for i in range(1, 13): #for e in mfcc:
        to_append += f' {np.mean(mfcc[i]):5.3f}' #f' {np.mean(e):5.3f}'
        
    # Split string into a list and insert track name
    arguments = to_append.split()
    arguments.insert(0, track)
    
    # Write to data.csv
    file = open(output_csv, 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(arguments)

### Alternative feature extractor with more features

In [ ]:
def extractTrackDataAndWriteToCSV_2(y, sr, output_csv):

    #----------------------------------------------------------------------------
    #-- Preperations --#
    hop_length = 512
    
    #seperate harmonics from percussives
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    # Musical-key-extractor by jackmcarthur (git-project)
    keyextract = Tonal_Fragment(y_harmonic, sr=sr)   
    # Separate a complex-valued spectrogram into its magnitude and phase
    S, phase = librosa.magphase(librosa.stft(y))
    # Compute onset strengths for tempo estimation purposes
    oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
    
    #----------------------------------------------------------------------------
    #-- Extraction --#    
    
    # Extract tempo - beats per minute
    tempo = float(librosa.beat.tempo(onset_envelope=oenv, sr=sr))
    
    # Extract onsets and devide by length to get rate
    onset = librosa.onset.onset_detect(y=y, sr=sr)
    onset_rate = (np.sum(onset) / np.size(onset))
    
    # Extract onset strength
    res = librosa.onset.onset_strength(y=y, sr=sr)
    onset_strength_mean = np.mean(res)
    onset_strength_std = np.std(res)
    
    # Extract zero crossings rate
    res = librosa.feature.zero_crossing_rate(y=y)
    zcr_mean = np.mean(res)
    zcr_std = np.std(res)
    
    # Extract spectral centroid
    res = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_centroid_mean = np.mean(res)
    spec_centroid_std = np.std(res)
    
    # Extract spectral contrast
    res = librosa.feature.spectral_contrast(y=y, sr=sr)
    spec_contrast_mean = np.mean(res) 
    spec_contrast_std = np.std(res) 
    
    # Extract spectral rolloff
    res = librosa.feature.spectral_rolloff(y=y, sr=sr)
    spec_rolloff_mean = np.mean(res)  
    spec_rolloff_std = np.std(res) 
    
    # Extract spectral_bandwidth -
    res = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_bandwidth_mean = np.mean(res)  
    spec_bandwidth_std = np.std(res)  
    
    # Extract spectral_flatness -
    res = librosa.feature.spectral_flatness(y=y)
    spectral_flatness_mean = np.mean(res)
    spectral_flatness_std = np.std(res)
    
    # Extract tonnetz - 
    tonnetz = librosa.feature.tonnetz(y=y, sr=sr, chroma=None)
    
    # Extract MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    
    #----------------------------------------------------------------------------
    #-- Processing --# 
    
    # Appends mean values of all arrays of data
    to_append = f'{tempo:5.3f} {onset_rate:5.3f}'
    
    to_append += f' {onset_strength_mean:5.3f} {onset_strength_std:5.3f}'
    to_append += f' {zcr_mean:5.3f} {zcr_std:5.3f}'
    to_append += f' {spec_centroid_mean:5.3f} {spec_centroid_std:5.3f}'
    to_append += f' {spec_contrast_mean:5.3f} {spec_contrast_std:5.3f}'
    to_append += f' {spec_rolloff_mean:5.3f} {spec_rolloff_std:5.3f}'
    to_append += f' {spec_bandwidth_mean:5.3f} {spec_bandwidth_std:5.3f}'
    to_append += f' {spectral_flatness_mean:8.6f} {spectral_flatness_std:8.6f}'
    
    # Add tonnetz
    for t in tonnetz:
        to_append += f' {np.mean(t):5.3f}'
        to_append += f' {np.std(t):5.3f}'
        
    # Add chroma tones
    keyextract.chroma_max = max(keyextract.chroma_vals)
    for key, chrom in keyextract.keyfreqs.items():
        to_append += f' {(chrom / keyextract.chroma_max):5.3f}'
        
    # Add mfccs
    for i in range(1, 13): #for e in mfcc:
        to_append += f' {np.mean(mfcc[i]):5.3f}' #f' {np.mean(e):5.3f}'
        to_append += f' {np.std(mfcc[i]):5.3f}' #f' {np.std(e):5.3f}'
        
    # Split string into a list and insert track name
    arguments = to_append.split()
    arguments.insert(0, track)
    
    # Write to data.csv
    file = open(output_csv, 'a', newline='')
    with file:
        writer = csv.writer(file)
        writer.writerow(arguments)

### Plot utility

In [ ]:
def fig_ax(figsize=(15, 5), dpi=150):
    """Return a (matplotlib) figure and ax objects with given size."""
    return plt.subplots(figsize=figsize, dpi=dpi)

### Laplacian Segmentation

In [ ]:
# Code source: Brian McFee
# License: ISC
# Eited by me as part of my master thesis: Victor Tideman

# implements the laplacian segmentation method of `McFee and Ellis, 
# 2014 <http://bmcfee.github.io/papers/ismir2014_spectral.pdf>`_, 
# with a couple of minor stability improvements.

def laplacian(y, sr, nr_klusters=6, track_name=""):
    
    # Next, we'll compute and plot a log-power CQT
    BINS_PER_OCTAVE = 12 * 3
    N_OCTAVES = 7
    C = librosa.amplitude_to_db(np.abs(librosa.cqt(y=y, sr=sr, bins_per_octave=BINS_PER_OCTAVE, n_bins=N_OCTAVES * BINS_PER_OCTAVE)), ref=np.max)

    # To reduce dimensionality, we'll beat-synchronous the CQT
    tempo, beats = librosa.beat.beat_track(y=y, sr=sr, trim=False)
    Csync = librosa.util.sync(C, beats, aggregate=np.median)

    # For plotting purposes, we'll need the timing of the beats
    # we fix_frames to include non-beat frames 0 and C.shape[1] (final frame)
    beat_times = librosa.frames_to_time(librosa.util.fix_frames(beats, x_min=0), sr=sr)

    # Let's build a weighted recurrence matrix using beat-synchronous CQT(Equation 1)
    # width=3 prevents links within the same bar
    # mode='affinity' here implements S_rep (after Eq. 8)
    R = librosa.segment.recurrence_matrix(Csync, width=3, mode='affinity', sym=True)

    # Enhance diagonals with a median filter (Equation 2)
    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, 7))
    
    # Now let's build the sequence matrix (S_loc) using mfcc-similarity
    # sigma to be the median distance between successive beats.
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
    Msync = librosa.util.sync(mfcc, beats)
    path_distance = np.sum(np.diff(Msync, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance / sigma)
    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)
    
    # And compute the balanced combination (Equations 6, 7, 9)
    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)
    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)
    A = mu * Rf + (1 - mu) * R_path
    
    # Now let's compute the normalized Laplacian (Eq. 10)
    L = scipy.sparse.csgraph.laplacian(A, normed=True)

    # and its spectral decomposition
    evals, evecs = scipy.linalg.eigh(L)
    
    # Apply a median filter to help smooth over small discontinuities and clean up.
    evecs = scipy.ndimage.median_filter(evecs, size=(9, 1))

    # cumulative normalization is needed for symmetric normalize laplacian eigenvectors
    Cnorm = np.cumsum(evecs**2, axis=1)**0.5

    # Use the first k normalized eigenvectors.
    k = nr_klusters
    X = evecs[:, :k] / Cnorm[:, k-1:k]

    # Let's use these k components to cluster beats into segments (Algorithm 1)
    KM = sklearn.cluster.KMeans(n_clusters=k)
    seg_ids = KM.fit_predict(X)
    colors = plt.get_cmap('Paired', k)

    # Locate segment boundaries from the label sequence
    bound_beats = 1 + np.flatnonzero(seg_ids[:-1] != seg_ids[1:])
    # Count beat 0 as a boundary
    bound_beats = librosa.util.fix_frames(bound_beats, x_min=0)
    # To avoid array out of index issue in some cases <- quick fix
    bound_beats = bound_beats[:-1]
    # Compute the segment label for each boundary
    bound_segs = list(seg_ids[bound_beats])
    # Convert beat indices to frames
    bound_frames = beats[bound_beats]
    # Make sure we cover to the end of the track
    bound_frames = librosa.util.fix_frames(bound_frames, x_min=None, x_max=C.shape[1]-1)
    # Convert frames to time
    bound_times = librosa.frames_to_time(bound_frames)
    
    # And plot the final segmentation over original CQT
    #freqs = librosa.cqt_frequencies(n_bins=C.shape[0], fmin=librosa.note_to_hz('C1'), bins_per_octave=BINS_PER_OCTAVE)
    #fig, ax = plt.subplots()
    #librosa.display.specshow(C, y_axis='cqt_hz', sr=sr, bins_per_octave=BINS_PER_OCTAVE, x_axis='time', ax=ax)

    #for interval, label in zip(zip(bound_times, bound_times[1:]), bound_segs):
    #    ax.add_patch(patches.Rectangle((interval[0], freqs[0]), interval[1] - interval[0], freqs[-1], facecolor=colors(label), alpha=0.50))
        
    return bound_times, bound_segs, nr_klusters


### Extraction point selection

In [ ]:
def extractStartingPoint(y, sr, trackpath):
    
    # Compute segments/timestamps where change is detected using laplacian method
    bound_times, bound_segs, nr_klusters = laplacian(y=y, sr=sr, nr_klusters=6, track_name=track)
    
    # Hacky fix to issue of some breakpoints being after the track
    duration = librosa.get_duration(y=y, sr=sr)
    original_size = bound_times.size
    bound_times = bound_times[(bound_times < duration)]
    new_size = bound_times.size
    nr_remove = original_size - new_size
    bound_segs = bound_segs[:-nr_remove]
    
    # Initialise
    selected_breakpoint = None

    # Get number of breakpoints (one less segments in total, segments exist between breakpoints)
    nr_breakpoints = bound_times.size
    
    # For tracing: print potential breakpoints
    #print(f'Potential breakpoints: {bound_times}')

    # Ignore intro and outro (they can be in the same cluster)
    intro_idx = bound_segs[0]
    # outro_idx = bound_segs[-1]

    # Create list for storing potential segments
    potential_segments = []

    # Add every segment that is not too short or of the same index as intro_idx or outro_idx
    # to a list of potential segments to use
    for i in range(0, nr_breakpoints - 1):
        seg_id = bound_segs[i]
        if (seg_id != intro_idx):

            # Get duration of a segment
            segment_duration = bound_times[i+1] - bound_times[i]

            # Make sure segment is longer than minimum duration
            if (segment_duration > float(min_duration)):
                potential_segments.append(bound_times[i])

    # make a numpy array
    segment_startpoints = np.array(potential_segments)
    nr_seg = segment_startpoints.size

    # Unless only 1 segment remains, compare remaining segments to select the most "expressable" segment
    if (nr_seg > 1):
        score_list = []

        #print(segment_startpoints)
        
        # Loop through the promising segments
        for segment_start in segment_startpoints:
            # We want a segment that is active in all frequency bands (base, mid and percussives)
            #print(f'load_segment at: {segment_start}')
            y_seg, sr = librosa.load(trackpath, mono=True, sr=44100, offset=segment_start, duration=extract_window_duration)
            S, phase = librosa.magphase(librosa.stft(y_seg))
            rms = np.mean(librosa.feature.rms(S=S))

            # Add segment value to the array
            score_list.append(rms)

        # Pick the segment with the largest value in the array
        selected_breakpoint = segment_startpoints[np.argmax(np.array(score_list))]

    # One or less viable breakpoints
    else:
        if (segment_startpoints.size != 0):
            selected_breakpoint = segment_startpoints[0]
        else:
            selected_breakpoint = 0
            
    return selected_breakpoint

# Extract features from tracks into a CSV file

## Create Header for CSV file

In [ ]:
# Create headers
header = 'track_name bpm onset_rate zcr spctrl_cent spctrl_cont spctrl_roll spctrl_band' 

# Add tonnetz to header
for i in range(1, 7):
    header += f' tonnetz_mean{i}'
    
# Add tones to header
pitches = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
for i in range(0, 12):
    header += f' {pitches[i]}'

# Add mfccs to header
for i in range(1, 13):
    header += f' mfcc_mean{i+1}'
    
header = header.split()
print(header)

In [ ]:
header_large = 'track_name bpm onset_rate' 
header_large += ' onset_strength_mean onset_strength_std'
header_large += ' zcr_mean zcr_std'
header_large += ' spec_cent_mean spec_cent_std'
header_large += ' spec_contrast_mean spec_contrast_std'
header_large += ' spec_rolloff_mean spec_rolloff_std'
header_large += ' spec_bandwidth_mean spec_bandwidth_std'
header_large += ' spectral_flatness_mean spectral_flatness_std'
    
# Add tonnetz to header
for i in range(1, 7):
    header_large += f' tonnetz_mean_{i}'
    header_large += f' tonnetz_std_{i}'
    
# Add tones to header
pitches = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
for i in range(0, 12):
    header_large += f' {pitches[i]}'

# Add mfccs to header
for i in range(1, 13):
    header_large += f' mfcc_mean_{i+1}'
    header_large += f' mfcc_std_{i+1}'
    
header_large = header_large.split()
print(header_large)
print(len(header_large))

## Structure analysis. Find segment of song to use

In [ ]:
# Music folder path
rootfolder = "track_library"

In [ ]:
# Set a minumum segment duration
extract_window_duration = 30
min_duration = 10

In [ ]:
#Path to directory with tracks
lst = list(set(os.listdir(f'{rootfolder}')) - {'desktop.ini', '.ipynb_checkpoints'})

# List of breakpoints
startingpoints = []

# Select a breakpoint for each track from where extraction should start
for track in lst:
    
    # Print what track and load it
    print(f'Analysing: {track}')  
    trackpath = f'{rootfolder}/{track}'
    y, sr = librosa.load(trackpath, mono=True, sr=None, duration=None)
    
    # Select a breakpoint
    selected_breakpoint = extractStartingPoint(y=y, sr=sr, trackpath=trackpath)

    # Print the selected breakpoint
    #print(f'Selected breakpoint at: {selected_breakpoint:5.2f} seconds into the track')
    startingpoints.append(selected_breakpoint)

## Extract features

In [ ]:
# Calcualte features and store in csv file
file_namee = 'data_69.csv'
file = open(file_namee, 'w', newline='')
with file:
    writer = csv.writer(file)
    #writer.writerow(header)
    writer.writerow(header_large)

# Generate per track
#for track in lst:
for i in range (0, len(lst)):
    trackpath = f'{rootfolder}/{lst[i]}'
    track = lst[i]
    y, sr = librosa.load(trackpath, mono=True, sr=None, offset=startingpoints[i], duration=30)
    
    # Extract data
    extractTrackDataAndWriteToCSV_2(y, sr, file_namee)

In [ ]:
# Read data from data.csv and display it
data = pd.read_csv('data_69.csv') # data = pd.read_csv('data.csv')
InitialValuesTable = copy.copy(data)
data_track_names = data['track_name']
data.drop(['track_name'], axis=1, inplace=True)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler
scaler = StandardScaler()

In [ ]:
# Standardise the data 
data[data.columns] = scaler.fit_transform(data[data.columns])

## Scaled Table | used as input for DR techniques

In [ ]:
# PRINT THE STANDARDISED DATA
#data

## Inital values linked with track names

In [ ]:
#InitialValuesTable

In [ ]:
id_track_list = np.empty(shape=(0, 2))
for i in range(0, data_track_names.shape[0]):
    #print(f'{i}\t{data_track_names[i]}')
    id_track_list = np.append(id_track_list, np.array([[i, data_track_names[i]]]), axis=0)


# DR - Explore extracted data

## T-SNE

In [ ]:
# TSNE
X = data.copy()
X_embedded = sklearn.manifold.TSNE(n_components=2, perplexity=5, learning_rate=200, init='pca').fit_transform(X)

# Create empty array
categories = np.zeros(X.shape[0])

groups = np.array(['Outliers','Sub-cluster 1','Sub-cluster 2','Sub-cluster 3','Sub-cluster 4','Sub-cluster 5','Sub-cluster 6',
                   'Sub-cluster 7','Sub-cluster 8','Sub-cluster 9','Sub-cluster 10','Sub-cluster 11'])
colormap = np.array(['black','darkred','green','blue','yellow','cyan','purple','orangered','pink','lime','sienna','teal'])
for id_track in id_track_list:
    idx = int(id_track[0])
    name = id_track[1]
    if (str(name[:1]) == "x"):
        categories[idx] = 0
    elif (str(name[:3]) == "c1_"):
        categories[idx] = 1
    elif (str(name[:3]) == "c2_"):
        categories[idx] = 2
    elif (str(name[:3]) == "c3_"):
        categories[idx] = 3
    elif (str(name[:3]) == "c4_"):
        categories[idx] = 4
    elif (str(name[:3]) == "c5_"):
        categories[idx] = 5
    elif (str(name[:3]) == "c6_"):
        categories[idx] = 6
    elif (str(name[:3]) == "c7_"):
        categories[idx] = 7
    elif (str(name[:3]) == "c8_"):
        categories[idx] = 8
    elif (str(name[:3]) == "c9_"):
        categories[idx] = 9
    elif (str(name[:3]) == "c10"):
        categories[idx] = 10
    elif (str(name[:3]) == "c11"):
        categories[idx] = 11
    

plt.scatter(X_embedded[:,0], X_embedded[:,1], s=150, c=colormap[categories.astype(int)])

# Add Legend
items = []
for i in range(0,groups.size):
    items.append(Line2D([0], [0], marker='o', color='whitesmoke', label=groups[i], markerfacecolor=colormap[i], markersize=10))
    plt.legend(handles=items)
    
# Add annotations
for i in range(0, data_track_names.shape[0]):
    plt.text(X_embedded[i,0]+2.3, X_embedded[i,1]-1.5, (id_track_list[i][1])[:-4] , horizontalalignment='left', size=12, color='black', weight='semibold')

plt.title('T-SNE')
plt.show()

In [ ]:
X = data.copy()

# Perform DR down to 3D 
X_embedded = sklearn.manifold.TSNE(n_components=3, perplexity=5, learning_rate=10, init='pca').fit_transform(X)

# creating a list of column names
column_values = ['x', 'y', 'z']
  
# creating the dataframe
df = pd.DataFrame(data = X_embedded, columns = column_values)

trace = go.Scatter3d(x=df.x, y=df.y, z=df.z, mode='markers', marker=dict(size=10, color=colormap[categories.astype(int)], opacity=0.7))
data_ = [trace]

#trace_0 = go.Scatter3d(x=(df.x).iloc[0:11], y=(df.y).iloc[0:11], z=(df.z).iloc[0:11], mode='markers', marker=dict(size=10, color=colormap[0], opacity=0.75), name='Outliers')
#trace_1 = go.Scatter3d(x=(df.x).iloc[11:16], y=(df.y).iloc[11:16], z=(df.z).iloc[11:16], mode='markers', marker=dict(size=10, color=colormap[1], opacity=0.75), name='Sub Cluster 1')
#trace_2 = go.Scatter3d(x=(df.x).iloc[16:22], y=(df.y).iloc[16:22], z=(df.z).iloc[16:22], mode='markers', marker=dict(size=10, color=colormap[2], opacity=0.75), name='Sub Cluster 2')
#trace_3 = go.Scatter3d(x=(df.x).iloc[22:29], y=(df.y).iloc[22:29], z=(df.z).iloc[22:29], mode='markers', marker=dict(size=10, color=colormap[3], opacity=0.75), name='Sub Cluster 3')
#trace_4 = go.Scatter3d(x=(df.x).iloc[31:41], y=(df.y).iloc[31:41], z=(df.z).iloc[31:41], mode='markers', marker=dict(size=10, color=colormap[4], opacity=0.75), name='')
#trace_5 = go.Scatter3d(x=(df.x).iloc[41:51], y=(df.y).iloc[41:51], z=(df.z).iloc[41:51], mode='markers', marker=dict(size=10, color=colormap[5], opacity=0.75), name='')
#trace_6 = go.Scatter3d(x=(df.x).iloc[51:61], y=(df.y).iloc[51:61], z=(df.z).iloc[51:61], mode='markers', marker=dict(size=10, color=colormap[6], opacity=0.75), name='')
#trace_7 = go.Scatter3d(x=(df.x).iloc[61:71], y=(df.y).iloc[61:71], z=(df.z).iloc[61:71], mode='markers', marker=dict(size=10, color=colormap[7], opacity=0.75), name='')
#trace_8 = go.Scatter3d(x=(df.x).iloc[71:81], y=(df.y).iloc[71:81], z=(df.z).iloc[71:81], mode='markers', marker=dict(size=10, color=colormap[8], opacity=0.75), name='')
#trace_9 = go.Scatter3d(x=(df.x).iloc[81:91], y=(df.y).iloc[81:91], z=(df.z).iloc[81:91], mode='markers', marker=dict(size=10, color=colormap[9], opacity=0.75), name='')
#trace_10 = go.Scatter3d(x=(df.x).iloc[91:101], y=(df.y).iloc[91:101], z=(df.z).iloc[91:101], mode='markers', marker=dict(size=10, color=colormap[10], opacity=0.75), name='')
#data_ = [trace_0, trace_1,  trace_2,  trace_3,]# trace_4, trace_5, trace_6, trace_7, trace_8, trace_9, trace_10]



layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
fig = go.Figure(data=data_, layout=layout)
fig.update_layout(autosize=False, width=800, height=800,)
fig.show()

## PaCMAP

In [ ]:
#pacmap
import pacmap

X2 = data.copy()

X2_embedded = pacmap.PaCMAP(n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0)
X2_transformed = X2_embedded.fit_transform(X2.values, init='pca')

# Create empty array
categories = np.zeros(X.shape[0])

groups = np.array(['Outliers','Sub-cluster 1','Sub-cluster 2','Sub-cluster 3','Sub-cluster 4','Sub-cluster 5','Sub-cluster 6',
                   'Sub-cluster 7','Sub-cluster 8','Sub-cluster 9','Sub-cluster 10','Sub-cluster 11'])
colormap = np.array(['black','darkred','green','blue','yellow','cyan','purple','orangered','pink','lime','sienna','teal'])
for id_track in id_track_list:
    idx = int(id_track[0])
    name = id_track[1]
    if (str(name[:1]) == "x"):
        categories[idx] = 0
    elif (str(name[:3]) == "c1_"):
        categories[idx] = 1
    elif (str(name[:3]) == "c2_"):
        categories[idx] = 2
    elif (str(name[:3]) == "c3_"):
        categories[idx] = 3
    elif (str(name[:3]) == "c4_"):
        categories[idx] = 4
    elif (str(name[:3]) == "c5_"):
        categories[idx] = 5
    elif (str(name[:3]) == "c6_"):
        categories[idx] = 6
    elif (str(name[:3]) == "c7_"):
        categories[idx] = 7
    elif (str(name[:3]) == "c8_"):
        categories[idx] = 8
    elif (str(name[:3]) == "c9_"):
        categories[idx] = 9
    elif (str(name[:3]) == "c10"):
        categories[idx] = 10
    elif (str(name[:3]) == "c11"):
        categories[idx] = 11

# Scatter
plt.scatter(X2_transformed[:,0], X2_transformed[:,1], s=150, c=colormap[categories.astype(int)])

# Add Legend
items = []
for i in range(0, groups.size):
    items.append(Line2D([0], [0], marker='o', color='whitesmoke', label=groups[i], markerfacecolor=colormap[i], markersize=10))
    plt.legend(handles=items)
    
# Add annotations
for i in range(0, data_track_names.shape[0]):
    plt.text(X2_transformed[i,0]+0.05, X2_transformed[i,1]-0.04, (id_track_list[i][1])[:-4], horizontalalignment='left', size=12, color='black', weight='semibold')
    
plt.title('PaCMAP')
plt.show()

In [ ]:
X2 = data.copy()

# Perform DR down to 3D
X2_embedded = pacmap.PaCMAP(n_components=3, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0)
X2_transformed = X2_embedded.fit_transform(X2.values, init='pca')

# creating a list of column names
column_values = ['x', 'y', 'z']
  
# creating the dataframe
df = pd.DataFrame(data = X2_transformed, columns = column_values)

trace = go.Scatter3d(x=df.x, y=df.y, z=df.z, mode='markers', marker=dict(size=10, color=colormap[categories.astype(int)], opacity=0.8))
data_ = [trace]
layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
fig = go.Figure(data=data_, layout=layout)
fig.update_layout(autosize=False, width=800, height=800)
fig.show()

# Similiraty Metric

In [ ]:
# = X_embedded.copy()
X = X2_transformed.copy()
X = 

# Current song index
current_song_idx = 0

similarity_metrics = []
# Compute distances from selected("playing") track to the others
for i in range(0, X.shape[0]):
    dist = np.linalg.norm(X[current_song_idx] - X[i])
    similarity_metrics.append([id_track_list[i][1], dist])
    
similarity_metrics.sort(key=lambda x: x[1])

for item in similarity_metrics:
    print ("Track: ",item[0], "\tDistance: ", item[1])

In [ ]:
# number of tracks / size of embedded array
X_size = X.shape[0]

sim_matrix = np.zeros(shape=(0, X_size))

# Comute 2d array of distances
for i in range(0, X.shape[0]):
    similarity_metrics = []
    for j in range(0, X.shape[0]):
        dist = np.linalg.norm(X[i] - X[j])
        similarity_metrics.append(dist)
    sim_matrix = np.append(sim_matrix, np.array([similarity_metrics]), axis=0)

In [ ]:
fig, ax = plt.subplots()

ax.matshow(sim_matrix)

for (i, j), z in np.ndenumerate(sim_matrix):
    ax.text(j, i, '{:0.1f}'.format(z), ha='center', va='center', bbox=dict(boxstyle='round', facecolor='white', edgecolor='0.3'))
    
ax.set_ylabel('tracks')
ax.set_xlabel('tracks')
ax.set_title('Distances between tracks')

plt.show()

# EVALUATION

## DEAM Dataset on AROUSAL and VALENCE with my extracted features

### Extract annotations and what quarter 

In [ ]:
# Deam anotaions
file_1 = r"deam\static_annotations_averaged_songs_1_2000.csv"
file_2 = r"deam\static_annotations_averaged_songs_2000_2058.csv"
annotations = pd.read_csv(file_1)
    
# Remove unneccesay columns
col_info = list(enumerate(annotations.columns))
annotations.drop(columns=[col_info[2][1]], inplace=True)  # Remove valance_std

col_info = list(enumerate(annotations.columns))
annotations.drop(columns=[col_info[3][1]], inplace=True)  # Remove arousal_std

# Rename columns
annotations.rename(columns = {' valence_mean':'valence'}, inplace = True)
annotations.rename(columns = {' arousal_mean':'arousal'}, inplace = True)

# Mean Normalize values (center around 0)
annotations['valence'] -= 5
annotations['arousal'] -= 5

def feeling_map(t):
    if t['valence'] >= 0 and t['arousal'] >= 0:
        return 0 
    if t['valence'] >= 0 and t['arousal'] < 0:
        return 1
    if t['valence'] < 0 and t['arousal'] >= 0:
        return 2 
    if t['valence'] < 0 and t['arousal'] < 0:
        return 3

# Add a new column
annotations["Feeling"] = annotations.apply(feeling_map, axis=1)

High-Valance-High-Arousal (0, "Happy")  
High-Valance-Low-Arousal (1, "Calm")  
Low-Valance-High-Arousal (2, "Angry")  
Low-Valance-Low-Arousal (3, "Sad") 

In [ ]:
annotations

### Extract features from the deam dataset

In [ ]:
#Path to directory with tracks
excerpts = list(set(os.listdir("deam/MEMD_audio/")) - {'desktop.ini', '.ipynb_checkpoints'})

# Calcualte features and store in csv file
file = open('my_deam_features_x.csv', 'w', newline='')
with file:
    writer = csv.writer(file) 
    writer.writerow(header_large)

number_ = 0
# Loop through the deam database
for excerpt in excerpts:
    songpath = f'deam/MEMD_audio/{excerpt}'
    track = excerpt
    y, sr = librosa.load(songpath, mono=True, sr=None)
    # Extract data 
    extractTrackDataAndWriteToCSV_2(y, sr, 'my_deam_features_x.csv')

In [ ]:
# Read data from data.csv and display it
deam_data = pd.read_csv('my_deam_features_69.csv')
InitialValuesTable = copy.copy(deam_data)
trackNames = deam_data['track_name']

# Preprocess data
# Reformat from .mp3(string) to int
for i in range (0, deam_data.shape[0]):            
    track_id = int(deam_data.at[i, 'track_name'].replace('.mp3',''))
    deam_data.at[i, 'track_name'] = track_id

# Cast column to int in order to sort it
deam_data = deam_data.astype({'track_name': 'int32'})

# Sort in order to align excerpts with the annotated data
deam_data.sort_values(by=['track_name'], inplace=True)

# Drop any tracks with id > 2000
deam_data = deam_data[deam_data['track_name'] <= 2000]

# Drop the 'track_name' column
deam_data.drop(['track_name'], axis=1, inplace=True)

In [ ]:
deam_data

In [ ]:
deam_data_valence = annotations['valence']
deam_data_arousal = annotations['arousal']

### Regression - Valence and Arousal - Seperate models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = deam_data.copy()
Y_val = deam_data_valence
Y_aro = deam_data_arousal

scaler = StandardScaler()
X_scale = scaler.fit_transform(X)

# Split into train and test sets with valence as target
X_train_valence, X_val_and_test_valence, Y_train_valence, Y_val_and_test_valence = train_test_split(X_scale, Y_val, test_size=0.3)
X_val_valence, X_test_valence, Y_val_valence, Y_test_valence =  train_test_split(X_val_and_test_valence, Y_val_and_test_valence, test_size=0.5)

# Split into train and test sets with arousal as target
X_train_arousal, X_val_and_test_arousal, Y_train_arousal, Y_val_and_test_arousal = train_test_split(X_scale, Y_aro, test_size=0.3)
X_val_arousal, X_test_arousal, Y_val_arousal, Y_test_arousal =  train_test_split(X_val_and_test_arousal, Y_val_and_test_arousal, test_size=0.5)

# X_train contains 70% of total dataset
print(X_train_valence.shape)
# X_test contains 30% of total dataset
print(X_test_valence.shape)

In [ ]:
# Train on valence
lr_valence = LinearRegression()
lr_valence.fit(X_train_valence, Y_train_valence)

# Train on arousal
lr_arousal = LinearRegression()
lr_arousal.fit(X_train_arousal, Y_train_arousal)

In [ ]:
# print the intercept
print("Valence intercept: ", lr_valence.intercept_)
print("Arousal intercept: ", lr_arousal.intercept_)

#Coefficients
coeff_val_df = pd.DataFrame(lr_valence.coef_, X.columns, columns=['Coefficient'])
#coeff_val_df
coeff_aro_df = pd.DataFrame(lr_arousal.coef_, X.columns, columns=['Coefficient'])
#coeff_aro_df

In [ ]:
pred_val = lr_valence.predict(X_test_valence)
pred_aro = lr_arousal.predict(X_test_arousal)

In [ ]:
plt.scatter(Y_test_valence, pred_val)
plt.title("Predicting valence scores with linear regression model")
plt.xlabel("valence")
plt.ylabel("Predicted Valence")
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

In [ ]:
from sklearn import metrics
print('Valence RMSE:', np.sqrt(metrics.mean_squared_error(Y_test_valence, pred_val)))

In [ ]:
plt.scatter(Y_test_arousal, pred_aro)
plt.title("Predicting arousal scores with linear regression model")
plt.xlabel("Arousal")
plt.ylabel("Predicted Arousal")
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

In [ ]:
print('Arousal RMSE:', np.sqrt(metrics.mean_squared_error(Y_test_arousal, pred_aro)))

In [ ]:
pred_valence_reg = lr_valence.predict(data)
pred_arousal_reg = lr_arousal.predict(data)

### Neural network - regression

In [ ]:
# import libraries of tensorflow to build neurral networks.
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler

In [ ]:
X = deam_data.copy()
Y_valence = deam_data_valence
Y_arousal = deam_data_arousal

In [ ]:
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)

### Neural network for predicting valence

In [ ]:
X_train, X_val_and_test, Y_train, Y_val_and_test =  train_test_split(X_scale, Y_valence, test_size=0.3)
X_val, X_test, Y_val, Y_test =  train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

In [ ]:
# Build model
model_1 = Sequential([
    Dense(32, input_shape=(X.shape[1],), activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear'),
])

In [ ]:
# Compile model
learning_rate = 0.01
model_1.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate), metrics=[tf.keras.metrics.RootMeanSquaredError()])
model_1.summary()

In [ ]:
# Save the weigths of the model
model_1.save_weights('model_1.h5')

In [ ]:
model_1.load_weights('model_1.h5') # reset
hist = model_1.fit(X_train, Y_train, validation_data=(X_val,Y_val), verbose=1, epochs=20)

In [ ]:
# step-4: Evalution
model_1.evaluate(X_test,Y_test)

In [ ]:
pred = model_1.predict(X_test)
plt.scatter(Y_test, pred)
plt.title("Predicting valence scores with neural network model")
plt.xlabel("Valence")
plt.ylabel("Predicted Valence")
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

In [ ]:
pred_valence = model_1.predict(data)

In [ ]:
plt.plot(hist.history['root_mean_squared_error'])
plt.plot(hist.history['val_root_mean_squared_error'])
plt.title('Model Performance')
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

### Neural network for predecting arousal

In [ ]:
X_train, X_val_and_test, Y_train, Y_val_and_test =  train_test_split(X_scale, Y_arousal, test_size=0.3)
X_val, X_test, Y_val, Y_test =  train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5)

print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

In [ ]:
# Build model
model_2 = Sequential([
    Dense(32, input_shape=(X.shape[1],), activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear'),
])

In [ ]:
# Compile model
learning_rate = 0.01
model_2.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate), metrics=[tf.keras.metrics.RootMeanSquaredError()])
model_2.summary()

In [ ]:
# Save the weigths of the model
model_2.save_weights('model_2.h5')

In [ ]:
model_2.load_weights('model_2.h5')
hist_2 = model_2.fit(X_train, Y_train, validation_data=(X_val,Y_val), verbose=1, epochs=20)

In [ ]:
# step-4: Evalution
model_2.evaluate(X_test,Y_test)

In [ ]:
pred = model_2.predict(X_test)
plt.scatter(Y_test, pred)
plt.title("Predicting arousal scores with neural network model")
plt.xlabel("Arousal")
plt.ylabel("Predicted Arousal")
plt.xlim(-3, 3)
plt.ylim(-3, 3)
plt.gca().set_aspect('equal', adjustable='box')
plt.draw()

In [ ]:
pred_arousal = model_2.predict(data)

In [ ]:
plt.plot(hist_2.history['root_mean_squared_error'])
plt.plot(hist_2.history['val_root_mean_squared_error'])
plt.title('Model Performance')
plt.ylabel('RMSE')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Read track names from library to estimate valance and arousal
data2 = pd.read_csv('data_69.csv')
data_track_names = data2['track_name']

# Turn into dataframe and add column name
data_track_names = data_track_names.to_frame()
data_track_names.columns = ['Track']

# Flatten the list of predicted valence and arousal
flat_pred_valence = [item for sublist in pred_valence for item in sublist]
flat_pred_arousal = [item for sublist in pred_arousal for item in sublist]


def feeling_map_1(t):
    if t['Valence-nn'] >= 0 and t['Arousal-nn'] >= 0:
        return 0 
    if t['Valence-nn'] >= 0 and t['Arousal-nn'] < 0:
        return 1
    if t['Valence-nn'] < 0 and t['Arousal-nn'] >= 0:
        return 2 
    if t['Valence-nn'] < 0 and t['Arousal-nn'] < 0:
        return 3
    
def feeling_map_2(t):
    if t['Valence-reg'] >= 0 and t['Arousal-reg'] >= 0:
        return 0 
    if t['Valence-reg'] >= 0 and t['Arousal-reg'] < 0:
        return 1
    if t['Valence-reg'] < 0 and t['Arousal-reg'] >= 0:
        return 2 
    if t['Valence-reg'] < 0 and t['Arousal-reg'] < 0:
        return 3

# Append regression and nerual netork results to dataframe
data_track_names['Valence-nn'] = flat_pred_valence
data_track_names['Arousal-nn'] = flat_pred_arousal
data_track_names["Feeling-nn"] = data_track_names.apply(feeling_map_1, axis=1)
data_track_names['Valence-reg'] = pred_valence_reg.tolist()
data_track_names['Arousal-reg'] = pred_arousal_reg.tolist()
data_track_names["Feeling-reg"] = data_track_names.apply(feeling_map_2, axis=1)
    
# Append values to original data as well
data2['valence-nn'] = flat_pred_valence
data2['arousal-nn'] = data_track_names['Arousal-nn']
data2['valence-reg'] = data_track_names['Valence-reg']
data2['arousal-reg'] = data_track_names['Arousal-reg']

High-Valance-High-Arousal (0, "Happy") - High-Valance-Low-Arousal (1, "Calm") - Low-Valance-High-Arousal (2, "Angry") - Low-Valance-Low-Arousal (3, "Sad") 

In [ ]:
# Sort by various values
#data_track_names.sort_values(by=['Valence-nn'])
data_track_names

### Colorcode according to feelings and do DR

In [ ]:
# TSNE
X = data.copy()
X_embedded = sklearn.manifold.TSNE(n_components=2, perplexity=8, learning_rate=200, init='pca').fit_transform(X)

# Create empty array
categories = np.zeros(X.shape[0])

groups = np.array(['0: Happy', '1: Calm', '2: Angry', '3: Sad'])
colormap = np.array(['forestgreen', 'lightblue', 'red', 'blue'])

feelings = data_track_names['Feeling-nn']
for i in range(0, X.shape[0]):
    feel = feelings.iloc[i]
    if (feel == 0):
        categories[i] = 0
    elif (feel == 1):
        categories[i] = 1
    elif (feel == 2):
        categories[i] = 2
    elif (feel == 3):
        categories[i] = 3

plt.scatter(X_embedded[:,0], X_embedded[:,1], s=150, c=colormap[categories.astype(int)])

# Add Legend
items = []
for i in range(0,groups.size):
    items.append(Line2D([0], [0], marker='o', color='whitesmoke', label=groups[i], markerfacecolor=colormap[i], markersize=10))
    plt.legend(handles=items)
    
# Add annotations
for i in range(0, data_track_names.shape[0]):
    plt.text(X_embedded[i,0]+2.3, X_embedded[i,1]-1.5, (id_track_list[i][1])[:-4] , horizontalalignment='left', size=12, color='black', weight='semibold')

plt.title('T-SNE - Neural Network')
plt.show()

In [ ]:
# TSNE
X = data.copy()
X_embedded = sklearn.manifold.TSNE(n_components=2, perplexity=8, learning_rate=200, init='pca').fit_transform(X)

# Create empty array
categories = np.zeros(X.shape[0])

groups = np.array(['0: Happy', '1: Calm', '2: Angry', '3: Sad'])
colormap = np.array(['forestgreen', 'lightblue', 'red', 'blue'])

feelings = data_track_names['Feeling-reg']
for i in range(0, X.shape[0]):
    feel = feelings.iloc[i]
    if (feel == 0):
        categories[i] = 0
    elif (feel == 1):
        categories[i] = 1
    elif (feel == 2):
        categories[i] = 2
    elif (feel == 3):
        categories[i] = 3

plt.scatter(X_embedded[:,0], X_embedded[:,1], s=150, c=colormap[categories.astype(int)])

# Add Legend
items = []
for i in range(0,groups.size):
    items.append(Line2D([0], [0], marker='o', color='whitesmoke', label=groups[i], markerfacecolor=colormap[i], markersize=10))
    plt.legend(handles=items)
    
# Add annotations
for i in range(0, data_track_names.shape[0]):
    plt.text(X_embedded[i,0]+2.3, X_embedded[i,1]-1.5, (id_track_list[i][1])[:-4] , horizontalalignment='left', size=12, color='black', weight='semibold')

plt.title('T-SNE - Linear Regression')
plt.show()

In [ ]:
# PaCMAP
X2 = data.copy()
X2_embedded = pacmap.PaCMAP(n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0)
X2_transformed = X2_embedded.fit_transform(X2.values, init='pca')

# Create empty array
categories = np.zeros(X2.shape[0])

groups = np.array(['0: Happy', '1: Calm', '2: Angry', '3: Sad'])
colormap = np.array(['forestgreen', 'lightblue', 'red', 'blue'])

feelings = data_track_names['Feeling-nn']
for i in range(0, X.shape[0]):
    feel = feelings.iloc[i]
    if (feel == 0):
        categories[i] = 0
    elif (feel == 1):
        categories[i] = 1
    elif (feel == 2):
        categories[i] = 2
    elif (feel == 3):
        categories[i] = 3

plt.scatter(X2_transformed[:,0], X2_transformed[:,1], s=150, c=colormap[categories.astype(int)])

# Add Legend
items = []
for i in range(0,groups.size):
    items.append(Line2D([0], [0], marker='o', color='whitesmoke', label=groups[i], markerfacecolor=colormap[i], markersize=10))
    plt.legend(handles=items)
    
# Add annotations
for i in range(0, data_track_names.shape[0]):
    plt.text(X2_transformed[i,0]+0.05, X2_transformed[i,1]-0.04, (id_track_list[i][1])[:-4], horizontalalignment='left', size=12, color='black', weight='semibold')

plt.title('PaCMAP - Neural Network')
plt.show()

In [ ]:
# PaCMAP
X2 = data.copy()
X2_embedded = pacmap.PaCMAP(n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0)
X2_transformed = X2_embedded.fit_transform(X2.values, init='pca')

# Create empty array
categories = np.zeros(X2.shape[0])

groups = np.array(['0: Happy', '1: Calm', '2: Angry', '3: Sad'])
colormap = np.array(['forestgreen', 'lightblue', 'red', 'blue'])

feelings = data_track_names['Feeling-reg']
for i in range(0, X.shape[0]):
    feel = feelings.iloc[i]
    if (feel == 0):
        categories[i] = 0
    elif (feel == 1):
        categories[i] = 1
    elif (feel == 2):
        categories[i] = 2
    elif (feel == 3):
        categories[i] = 3

plt.scatter(X2_transformed[:,0], X2_transformed[:,1], s=150, c=colormap[categories.astype(int)])

# Add Legend
items = []
for i in range(0,groups.size):
    items.append(Line2D([0], [0], marker='o', color='whitesmoke', label=groups[i], markerfacecolor=colormap[i], markersize=10))
    plt.legend(handles=items)
    
# Add annotations
for i in range(0, data_track_names.shape[0]):
    plt.text(X2_transformed[i,0]+0.05, X2_transformed[i,1]-0.04, (id_track_list[i][1])[:-4], horizontalalignment='left', size=12, color='black', weight='semibold')

plt.title('PaCMAP - Linear Regression')
plt.show()

### Classification with DEAM data set

Using my extracted features

In [ ]:
# Load data set
my_features = deam_data.copy()
feeling_label = annotations['Feeling']
print(my_features.shape)
print(feeling_label.shape)

In [ ]:
# Standardise the data 
my_features[my_features.columns] = scaler.fit_transform(my_features[my_features.columns])

In [ ]:
#my_features

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(my_features, feeling_label, test_size=0.3)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
train_score = knn.score(X_train, y_train)
test_score = knn.score(X_test, y_test)

# Round dwon the floats to two decimals
train_score_rounded = round(train_score, 2)
test_score_rounded = round(test_score, 2)

print("training_score: ",train_score_rounded," test_score: ", test_score_rounded)

Using DEAM's features

In [ ]:
#dfObj = pd.DataFrame(columns=['User_ID', 'UserName', 'Action'])
dfObj = pd.DataFrame()

for ids in annotations['song_id']:
    file = f'deam\\features\{ids}.csv'
    deam_features = pd.read_csv(file, sep = ';')
    deam_features.drop('frameTime', inplace=True,axis=1)
    deam_features = deam_features.mean(axis=0)
    dfObj = dfObj.append(deam_features, ignore_index=True)

In [ ]:
# Standardise the data 
dfObj[dfObj.columns] = scaler.fit_transform(dfObj[dfObj.columns])

In [ ]:
dfObj

In [ ]:
deam_features = dfObj.copy()
feeling_label = annotations['Feeling']
print(deam_features.shape)
print(feeling_label.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(deam_features, feeling_label, test_size=0.3)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
train_score = knn.score(X_train, y_train)
test_score = knn.score(X_test, y_test)
# Round dwon the floats to two decimals
train_score_rounded = round(train_score, 2)
test_score_rounded = round(test_score, 2)

print("training_score: ",train_score_rounded," test_score: ", test_score_rounded)

## 100 copies of a song with 10 incremental changes in 10 varying dimensions 

In [ ]:
# Variables
filename = 'eval_69.csv'
track = 'c1_1_ame.wav' # Full Intention - America (I Love America) (Full Length Vocal Mix)
track_path = 'track_library/c1_1_ame.wav'

# Load the track
y_eval, sr_eval = librosa.load(track_path, mono=True, sr=None, duration=None)

# select starting point for feature selection
startpoint = extractStartingPoint(y_eval, sr_eval, track_path)

# Reload track but only the selected segment
y_eval, sr_eval = librosa.load(track_path, mono=True, sr=None, offset=startpoint, duration=30)

print(f'Starting point selected at {startpoint} seconds into the track')

from audiomentations import (
    Compose,
    LowPassFilter,
    HighPassFilter,
)

# High pass filter
hipass = 20
augment = Compose([HighPassFilter(min_cutoff_freq=hipass, max_cutoff_freq=hipass, p = 1)])
y_eval = augment(samples=y_eval, sample_rate=sr_eval)

# low pass filter
lopass = 20000
augment = Compose([LowPassFilter(min_cutoff_freq=lopass, max_cutoff_freq=lopass, p = 1)])
y_eval = augment(samples=y_eval, sample_rate=sr_eval)

print(f'removed frequencies outside [20, 20000]')

In [ ]:
from audiomentations import (
    Compose,
    LowPassFilter,
    HighPassFilter,
    AddGaussianNoise,
    TanhDistortion,
)

from pedalboard import (
    Pedalboard,
    Reverb,
    Phaser,
    Chorus,
    Bitcrush,
    Gain,
)

#Select deature set: 0 = 39 feature data, 1 = 69 feature data
feature_set = 1

# Calcualte features and store in csv file
file = open(filename, 'w', newline='')
with file:
    writer = csv.writer(file)
    if (feature_set == 0):
        writer.writerow(header)
    else:
        writer.writerow(header_large)
    
track = f'Original'
if (feature_set == 0):
    extractTrackDataAndWriteToCSV(y_eval, sr_eval, filename)
else:
    extractTrackDataAndWriteToCSV_2(y_eval, sr_eval, filename)
    
# Dimension 1.1 - bpm up
steps = np.arange(1.02, 1.10, 0.02)
for i in range(0, 5):
    y_dim_1 = pyrb.time_stretch(y_eval, sr_eval, steps[i])
    track = f'bpm_up_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_1, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_1, sr_eval, filename)
    
# Dimension 1.2 - bpm down
steps = np.arange(0.98, 0.88, -0.02)
for i in range(0, 5):
    y_dim_1 = pyrb.time_stretch(y_eval, sr_eval, steps[i])
    track = f'bpm_down_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_1, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_1, sr_eval, filename)
    
print("Extracted dim 1/10")
    
# Dimension 2.1 - Pitch up
steps = np.arange(0.5, 3, 0.5)
for i in range(0, 5):
    y_dim_2 = pyrb.pitch_shift(y_eval, sr_eval, steps[i]) 
    track = f'pitch_up_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_2, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_2, sr_eval, filename)
    
# Dimension 2.2 - Pitch down
steps = np.arange(-0.5, -3, -0.5)
for i in range(0, 5):
    y_dim_2 = pyrb.pitch_shift(y_eval, sr_eval, steps[i]) 
    track = f'pitch_down_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_2, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_2, sr_eval, filename)
        
print("Extracted dim 2/10")
    
# Dimension 3 - High pass filter
steps = np.arange(30, 330, 30)
for i in range(0, 10):
    augment = Compose([HighPassFilter(min_cutoff_freq=steps[i], max_cutoff_freq=steps[i], p = 1)])
    y_dim_3 = augment(samples=y_eval, sample_rate=sr_eval)
    track = f'hpfilter_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_3, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_3, sr_eval, filename)
        
print("Extracted dim 3/10")

# Dimension 4 - low pass filter
steps = np.arange(19000, 14000, -500)
for i in range(0, 10):
    augment = Compose([LowPassFilter(min_cutoff_freq=steps[i], max_cutoff_freq=steps[i], p = 1)])
    y_dim_4 = augment(samples=y_eval, sample_rate=sr_eval)
    track = f'lpfilter_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_4, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_4, sr_eval, filename)
    
print("Extracted dim 4/10")

# Dimension 5 - Gaussion noise
steps = np.arange(0.01, 0.11 , 0.01)
for i in range(0, 10):
    augment = Compose([AddGaussianNoise(min_amplitude=steps[i], max_amplitude=steps[i], p=1)])
    y_dim_5 = augment(samples=y_eval, sample_rate=sr_eval)
    track = f'gauss_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_5, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_5, sr_eval, filename)
    
print("Extracted dim 5/10")

# Dimension 6 - TanhDistortion
steps = np.arange(0.35, 0.85, 0.05)
for i in range(0, 10):
    augment = Compose([TanhDistortion(min_distortion = steps[i], max_distortion = steps[i], p=1)])
    y_dim_6 = augment(samples=y_eval, sample_rate=sr_eval)
    track = f'tanh_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_6, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_6, sr_eval, filename)
        
print("Extracted dim 6/10")

# Dimension 7 - bit crush
steps = np.arange(7.75, 5.25, -0.25)
board = Pedalboard([Bitcrush()])
for i in range(0, 10):
    board[0].bit_depth = steps[i]
    y_dim_7 = board(y_eval, sr_eval)
    track = f'bitcrush_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_7, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_7, sr_eval, filename)
        
print("Extracted dim 7/10")

# Dimension 8 - chorus
steps = np.arange(1, 11, 1)
board = Pedalboard([Chorus()])
for i in range(0, 10):
    board[0].rate_hz = steps[i]
    #board[0].depth = i
    y_dim_8 = board(y_eval, sr_eval)
    track = f'Chorus_rate_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_8, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_8, sr_eval, filename)
    
print("Extracted dim 8/10")

# Dimension 9 - phaser
steps = np.arange(1, 11, 1)
board = Pedalboard([Phaser()])
for i in range(0, 10):
    board[0].rate_hz = steps[i]
    board[0].depth = 0.5
    y_dim_9 = board(y_eval, sr_eval)
    track = f'Phaser_rate_{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_9, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_9, sr_eval, filename)
    
print("Extracted dim 9/10")

# Dimension 10 - Gain
steps = np.arange(1, 11, 1)
board = Pedalboard([Gain()])
for i in range(0, 10):
    board[0].gain_db = steps[i]
    y_dim_9 = board(y_eval, sr_eval)
    track = f'gain{i+1}'
    if (feature_set == 0):
        extractTrackDataAndWriteToCSV(y_dim_9, sr_eval, filename)
    else:
        extractTrackDataAndWriteToCSV_2(y_dim_9, sr_eval, filename)

print("Extracted dim 10/10")

In [ ]:
for i in np.arange(30, 330, 30):
    print (i)

In [ ]:
# Plot
#eval_data = pd.read_csv('eval.csv')
eval_data = pd.read_csv('eval_69.csv')
eval_data_track_names = eval_data['track_name']
eval_data.drop(['track_name'], axis=1, inplace=True)

In [ ]:
# Standardise the data 
eval_data[eval_data.columns] = scaler.fit_transform(eval_data[eval_data.columns])

In [ ]:
# t-SNE
X = eval_data.copy()
X_embedded = sklearn.manifold.TSNE(n_components=2, perplexity=10, learning_rate=10, init='pca').fit_transform(X)

# Create empty array
categories = np.zeros(X.shape[0])

groups = np.array(['Original','BPM','Pitch','High Pass Filter','Low Pass Filter','Gaussion Noise','Tanh Distortion','Bit Crush','Chorus','Phaser','Gain'])
colormap = np.array(['black', 'darkred', 'green', 'blue', 'yellow', 'cyan', 'purple', 'orangered', 'gold', 'magenta', 'pink'])
for i in range(0, X.shape[0]):    
    if (i == 0):
        categories[i] = 0
    elif ( 1 <= i <= 10):
        categories[i] = 1
    elif ( 11 <= i <= 20):
        categories[i] = 2
    elif ( 21 <= i <= 30):
        categories[i] = 3
    elif ( 31 <= i <= 40):
        categories[i] = 4
    elif ( 41 <= i <= 50):
        categories[i] = 5
    elif ( 51 <= i <= 60):
        categories[i] = 6
    elif ( 61 <= i <= 70):
        categories[i] = 7
    elif ( 71 <= i <= 80):
        categories[i] = 8
    elif ( 81 <= i <= 90):
        categories[i] = 9
    elif ( 91 <= i <= 100):
        categories[i] = 10

# colorbar and annotations 
plt.scatter(X_embedded[:,0], X_embedded[:,1], s=100, c=colormap[categories.astype(int)])

# Add Legend
items = []
for i in range(0,groups.size):
    items.append(Line2D([0], [0], marker='o', color='whitesmoke', label=groups[i], markerfacecolor=colormap[i], markersize=10))
    plt.legend(handles=items)

# Add annotations
for i in range(0, eval_data_track_names.shape[0]):
    if(i==0):
        plt.text(X_embedded[i,0]+0.3, X_embedded[i,1]-0.3, eval_data_track_names[i], horizontalalignment='left', size=18, color='black', weight='semibold')
    elif(i%5==0): # else:
        plt.text(X_embedded[i,0]+0.3, X_embedded[i,1]-0.25, eval_data_track_names[i], horizontalalignment='left', size=13, color='black', weight='semibold')
        
plt.title('T-SNE')
plt.show()

In [ ]:
X = eval_data.copy()

# Perform DR down to 3D 
X_embedded = sklearn.manifold.TSNE(n_components=3, perplexity=10, learning_rate=10, init='random').fit_transform(X)

# creating a list of column names
column_values = ['x', 'y', 'z']
  
# creating the dataframe
df = pd.DataFrame(data = X_embedded, columns = column_values)

trace_0 = go.Scatter3d(x=(df.x).iloc[0:1], y=(df.y).iloc[0:1], z=(df.z).iloc[0:1], mode='markers', marker=dict(size=15, color=colormap[0], opacity=0.9), name='Original')
trace_1 = go.Scatter3d(x=(df.x).iloc[1:11], y=(df.y).iloc[1:11], z=(df.z).iloc[1:11], mode='markers', marker=dict(size=10, color=colormap[1], opacity=0.75), name='BPM')
trace_2 = go.Scatter3d(x=(df.x).iloc[11:21], y=(df.y).iloc[11:21], z=(df.z).iloc[11:21], mode='markers', marker=dict(size=10, color=colormap[2], opacity=0.75), name='Pitch')
trace_3 = go.Scatter3d(x=(df.x).iloc[21:31], y=(df.y).iloc[21:31], z=(df.z).iloc[21:31], mode='markers', marker=dict(size=10, color=colormap[3], opacity=0.75), name='High Pass Filter')
trace_4 = go.Scatter3d(x=(df.x).iloc[31:41], y=(df.y).iloc[31:41], z=(df.z).iloc[31:41], mode='markers', marker=dict(size=10, color=colormap[4], opacity=0.75), name='Low Pass Filter')
trace_5 = go.Scatter3d(x=(df.x).iloc[41:51], y=(df.y).iloc[41:51], z=(df.z).iloc[41:51], mode='markers', marker=dict(size=10, color=colormap[5], opacity=0.75), name='Gaussian Noise')
trace_6 = go.Scatter3d(x=(df.x).iloc[51:61], y=(df.y).iloc[51:61], z=(df.z).iloc[51:61], mode='markers', marker=dict(size=10, color=colormap[6], opacity=0.75), name='Tanh Distortion')
trace_7 = go.Scatter3d(x=(df.x).iloc[61:71], y=(df.y).iloc[61:71], z=(df.z).iloc[61:71], mode='markers', marker=dict(size=10, color=colormap[7], opacity=0.75), name='Bit Crush')
trace_8 = go.Scatter3d(x=(df.x).iloc[71:81], y=(df.y).iloc[71:81], z=(df.z).iloc[71:81], mode='markers', marker=dict(size=10, color=colormap[8], opacity=0.75), name='Chorus')
trace_9 = go.Scatter3d(x=(df.x).iloc[81:91], y=(df.y).iloc[81:91], z=(df.z).iloc[81:91], mode='markers', marker=dict(size=10, color=colormap[9], opacity=0.75), name='Phaser')
trace_10 = go.Scatter3d(x=(df.x).iloc[91:101], y=(df.y).iloc[91:101], z=(df.z).iloc[91:101], mode='markers', marker=dict(size=10, color=colormap[10], opacity=0.75), name='Gain')
data_ = [trace_0, trace_1, trace_2, trace_3, trace_4, trace_5, trace_6, trace_7, trace_8, trace_9, trace_10]

layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
fig = go.Figure(data=data_, layout=layout)
fig.update_layout(autosize=False, width=1000, height=800,)
fig.show()

In [ ]:
#pacmap
import pacmap

X2 =  eval_data.copy()
X2_embedded = pacmap.PaCMAP(n_components=2, n_neighbors=3, MN_ratio=3, FP_ratio=4)
X2_transformed = X2_embedded.fit_transform(X2.values, init='pca')

# Create empty array
categories = np.zeros(X.shape[0])

groups = np.array(['Original','BPM','Pitch','High Pass Filter','Low Pass Filter','Gaussion Noise','Tanh Distortion','Bit Crush','Chorus','Phaser', 'Gain'])
colormap = np.array(['black', 'darkred', 'green', 'blue', 'yellow', 'cyan', 'purple', 'orangered', 'gold', 'magenta', 'pink'])
for i in range(0, X.shape[0]):    
    if (i == 0):
        categories[i] = 0
    elif ( 1 <= i <= 10):
        categories[i] = 1
    elif ( 11 <= i <= 20):
        categories[i] = 2
    elif ( 21 <= i <= 30):
        categories[i] = 3
    elif ( 31 <= i <= 40):
        categories[i] = 4
    elif ( 41 <= i <= 50):
        categories[i] = 5
    elif ( 51 <= i <= 60):
        categories[i] = 6
    elif ( 61 <= i <= 70):
        categories[i] = 7
    elif ( 71 <= i <= 80):
        categories[i] = 8
    elif ( 81 <= i <= 90):
        categories[i] = 9
    elif ( 91 <= i <= 100):
        categories[i] = 10

# colorbar and annotations 
plt.scatter(X2_transformed[:,0], X2_transformed[:,1], s=100, c=colormap[categories.astype(int)])

# Add Legend
items = []
for i in range(0, groups.size):
    items.append(Line2D([0], [0], marker='o', color='whitesmoke', label=groups[i], markerfacecolor=colormap[i], markersize=10))
    plt.legend(handles=items)

# Add annotations
for i in range(0, eval_data_track_names.shape[0]):
    if(i==0):
        plt.text(X2_transformed[i,0]+0.1, X2_transformed[i,1]-0.2, eval_data_track_names[i], horizontalalignment='left', size=18, color='black', weight='semibold')
    elif(i%5==0): # else:
        plt.text(X2_transformed[i,0]+0.1, X2_transformed[i,1]-0.1, eval_data_track_names[i], horizontalalignment='left', size=13, color='black', weight='semibold')
        
plt.title('PaCMAP')
plt.show()

In [ ]:
X2 = eval_data.copy()

# Perform DR down to 3D
X2_embedded = pacmap.PaCMAP(n_components=3, n_neighbors=3, MN_ratio=3, FP_ratio=4)
X2_transformed = X2_embedded.fit_transform(X2.values, init='pca')

# creating a list of column names
column_values = ['x', 'y', 'z']
  
# creating the dataframe
df = pd.DataFrame(data = X2_transformed, columns = column_values)

trace_0 = go.Scatter3d(x=(df.x).iloc[0:1], y=(df.y).iloc[0:1], z=(df.z).iloc[0:1], mode='markers', marker=dict(size=15, color=colormap[0], opacity=0.9), name='Original')
trace_1 = go.Scatter3d(x=(df.x).iloc[1:11], y=(df.y).iloc[1:11], z=(df.z).iloc[1:11], mode='markers', marker=dict(size=10, color=colormap[1], opacity=0.75), name='BPM')
trace_2 = go.Scatter3d(x=(df.x).iloc[11:21], y=(df.y).iloc[11:21], z=(df.z).iloc[11:21], mode='markers', marker=dict(size=10, color=colormap[2], opacity=0.75), name='Pitch')
trace_3 = go.Scatter3d(x=(df.x).iloc[21:31], y=(df.y).iloc[21:31], z=(df.z).iloc[21:31], mode='markers', marker=dict(size=10, color=colormap[3], opacity=0.75), name='High Pass Filter')
trace_4 = go.Scatter3d(x=(df.x).iloc[31:41], y=(df.y).iloc[31:41], z=(df.z).iloc[31:41], mode='markers', marker=dict(size=10, color=colormap[4], opacity=0.75), name='Low Pass Filter')
trace_5 = go.Scatter3d(x=(df.x).iloc[41:51], y=(df.y).iloc[41:51], z=(df.z).iloc[41:51], mode='markers', marker=dict(size=10, color=colormap[5], opacity=0.75), name='Gaussian Noise')
trace_6 = go.Scatter3d(x=(df.x).iloc[51:61], y=(df.y).iloc[51:61], z=(df.z).iloc[51:61], mode='markers', marker=dict(size=10, color=colormap[6], opacity=0.75), name='Tanh Distortion')
trace_7 = go.Scatter3d(x=(df.x).iloc[61:71], y=(df.y).iloc[61:71], z=(df.z).iloc[61:71], mode='markers', marker=dict(size=10, color=colormap[7], opacity=0.75), name='Bit Crush')
trace_8 = go.Scatter3d(x=(df.x).iloc[71:81], y=(df.y).iloc[71:81], z=(df.z).iloc[71:81], mode='markers', marker=dict(size=10, color=colormap[8], opacity=0.75), name='Chorus')
trace_9 = go.Scatter3d(x=(df.x).iloc[81:91], y=(df.y).iloc[81:91], z=(df.z).iloc[81:91], mode='markers', marker=dict(size=10, color=colormap[9], opacity=0.75), name='Phaser')
trace_10 = go.Scatter3d(x=(df.x).iloc[91:101], y=(df.y).iloc[91:101], z=(df.z).iloc[91:101], mode='markers', marker=dict(size=10, color=colormap[10], opacity=0.75), name='Gain')
data_ = [trace_0, trace_1, trace_2, trace_3, trace_4, trace_5, trace_6, trace_7, trace_8, trace_9, trace_10]

layout = go.Layout(margin=dict(l=0, r=0, b=0, t=0))
fig = go.Figure(data=data_, layout=layout)
fig.update_layout(autosize=False, width=1000, height=800,)
fig.show()

# Extracting 30s

In [ ]:
import soundfile as sf

root_folder = "track_library"
lst_2 = list(set(os.listdir(f'{root_folder}')) - {'desktop.ini', '.ipynb_checkpoints'})

for i in range(0, len(startingpoints)):
    # Write out audio as 24bit PCM WAV
    track = lst_2[i]
    #print (i, track)  
    y, sr = librosa.load(f'track_library/{track}', mono=True, sr=None, offset=startingpoints[i], duration=30)
    #sf.write(f'tracks_30s/{track}', y, sr, subtype='PCM_24')